# EDA Business Insights

### Overall Demand & Revenue Snapshot

Before going deep into patterns, I want a high-level view of the business.
This helps answer very basic but important questions:

- How many orders are we handling overall?
- What does daily demand look like on average?
- How much revenue does this dataset represent?

This gives a baseline to compare everything else later.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

orders_df = pd.read_csv("../data/processed/order_history_cleaned.csv")

In [14]:
total_orders = orders_df.shape[0]
total_revenue = orders_df['total'].sum()
avg_daily_orders = orders_df.groupby('order_date').size().mean()
avg_daily_revenue = orders_df.groupby('order_date')['total'].sum().mean()

total_orders, total_revenue, avg_daily_orders, avg_daily_revenue

(21131,
 np.float64(14423379.76),
 np.float64(138.11111111111111),
 np.float64(94270.45594771243))